In [74]:
import random
import matplotlib.pyplot as plt
import seaborn
from sklearn import *
import numpy as np
import networkx as nx

import os

np.random.seed(42)

## Logistic Graph Modeling


### Objective:
To model the probability of a connection between two vertices in a graph using a logistic function.

### Equation:
\[
\text{prob\_of\_vertex}_{i,j} = \frac{c}{1 + \beta \exp(|i| + |j|)}
\]

Where:
- \( \text{prob\_of\_vertex}_{i,j} \) is the probability of a connection between vertices \( i \) and \( j \).
- \( c \) is the initial probability.
- \( \beta \) is a parameter that modulates the exponential term.

### Steps:
1. **Initialization**:
   - Set \( n \): Number of vertices.
   - Set \( c \): Initial probability (e.g., \( 1 \times 10^{-3} \)).
   - Set \( \beta \): A low value to start with.
  
2. **Logistic Regression**:
   - Use the given logistic function to compute the probability of a connection between vertices \( i \) and \( j \).

3. **Convergence**:
   - Monitor the model for convergence. The exact criterion for convergence isn't predefined.

4. **Parameter Estimation**:
   - Estimate \( c_{\text{hat}} \) and \( \beta_{\text{hat}} \).
   - Assess the goodness of fit of the estimations.

5. **Iterative Refinement**:
   - Incorporate more parameters based on the connectivity of the vertices.
   - For instance, for a vertex \( i \), consider all vertices connected to \( i \), and for vertex \( j \), consider all vertices connected to \( j \) and so on.
   - Repeat the logistic regression with increasing order \( p \) (up to \( p \leq 4 \)).

### Note:
The methodology seems to be an iterative approach to improve the modeling of vertex connectivity. The inclusion of more parameters based on connected vertices in step 5 suggests that the model may try to capture higher-order relationships or dependencies between vertices in the graph.

In [107]:
def plot_graph_from_adjacency(adj_matrix, filename, pos=None, title='graph'):
    G = nx.Graph()  # Initialize an undirected graph
    n = len(adj_matrix)

    # Add edges to the graph
    for i in range(n):
        for j in range(n):
            if adj_matrix[i][j] == 1:
                G.add_edge(i, j)

    # Draw the graph
    #fig = plt.figure()
    #plt.title(title)
    #nx.draw(G, with_labels=True, node_size=700, node_color="skyblue", font_size=15, font_weight='bold')
    #plt.show()
    fig = plt.figure()
    plt.title(title)
    nx.draw(G, pos=pos, with_labels=True, node_size=700, node_color="skyblue", font_size=15, font_weight='bold')
    
    # Save the figure as an image
    plt.savefig(filename, format='png')
    plt.close(fig)  # Close the figure to release resources
    return fig

def generate_random_graph(n, p):
    """Generate a random graph represented by an adjacency matrix.

    Parameters:
    - n: Number of vertices
    - p: Probability of an edge between any two vertices

    Returns:
    - adj_matrix: Adjacency matrix representing the graph
    """

    adj_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(i+1, n):  # This ensures an undirected graph
            if np.random.rand() < p:
                adj_matrix[i, j] = 1
                adj_matrix[j, i] = 1  # Symmetric entry for undirected graph

    return adj_matrix

In [129]:
# Define the graph as an adjcency matrix where n is the order of the graph
# Convention: Lines are vertex and columns are where the vertex is connected
def initialize_graph( n ):
    return np.zeros( ( n , n ) )

# Simple logistic regression function c / (1+beta*np.exp(sum_degrees))
def logistic_regression(c, beta, sum_degrees):
    num     = c
    denom   = 1 + beta * np.exp(-sum_degrees)
    return num / denom

# Returns a vector of degree of vertex i and a degree of each neighboor
# with a distance of p from the vertex i
def degree_vertex(adj_matrix, vertex, p):
    """Returns the degree of the vertex and degrees of neighbors within a distance of p."""
    n = len(adj_matrix)

    # Function to get neighbors of a given vertex
    def get_neighbors(v):
        return [i for i, x in enumerate(adj_matrix[v]) if x == 1]

    # Function to get degree of a given vertex
    def get_degree(v):
        return sum(adj_matrix[v])
        #return sum(adj_matrix[v]) / (n*(n-1)/2)

    # Base case for p=1 and p=0
    if p == 0:
        neighbors = get_neighbors(vertex)
        return [get_degree(vertex)]

    if p == 1:
        neighbors = get_neighbors(vertex)
        return [get_degree(vertex)] + [get_degree(neighbor) for neighbor in neighbors]

    # For p > 1
    visited = set([vertex])
    current_neighbors = get_neighbors(vertex)
    for _ in range(p-1):
        next_neighbors = []
        for v in current_neighbors:
            neighbors_of_v = get_neighbors(v)
            next_neighbors.extend([nv for nv in neighbors_of_v if nv not in visited])
            visited.add(v)
        current_neighbors = list(set(next_neighbors))

    return [get_degree(vertex)] + [get_degree(neighbor) for neighbor in current_neighbors]

def get_sum_degrees(graph, vertex, p=1):
    """Gets the sum of degrees for a vertex considering a distance p."""
    return sum(degree_vertex(graph, vertex, p))

def get_edge_logit(c, beta, sum_degrees, threshold):
    """Decides if an edge should be added based on the logistic regression output and a threshold."""
    val_log = logistic_regression(c, beta, sum_degrees)
    #return 1 if logistic_regression(c, beta, sum_degrees) >= threshold else 0
    return np.random.choice(np.arange(0, 2), p=[1-val_log, val_log])

# TODO: I have to change this function. It is not working properly. 
# The reason for that is because the sum of degrees is always a large number
# and this does not work well with the logistic regression.
def add_vertex(graph, c, beta, p, threshold, sigma=1):
    """Modified function to iterate over the graph's vertices and decide if an edge will be added."""
    n, m = graph.shape
    for i in range(n):
        for j in range(m):
            if i != j and graph[i,j] == 0:
                normalization = n * (n-1) /2
                sum_degrees_i = get_sum_degrees(graph, i, p) / (normalization)
                sum_degrees_j = get_sum_degrees(graph, j, p) / (normalization)
                sum_degrees = sum_degrees_i + sum_degrees_j + abs(np.random.normal(0, sigma))

                graph[i, j] = get_edge_logit(c, beta, sum_degrees, threshold)
                #print('Testing for:', i, j, logistic_regression(c, beta, sum_degrees), 'Value of edge', get_edge_logit(c, beta, sum_degrees, threshold))
    return graph

def check_convergence(current_graph, previous_graph, tolerance=0.01):
    """
    Checks if the graph has converged by comparing the current adjacency matrix
    with the one from the previous iteration.

    Args:
    - current_graph: The current adjacency matrix.
    - previous_graph: The adjacency matrix from the previous iteration.
    - tolerance: A small value. If the difference between the matrices (in terms of total number of edges)
                 is less than this value, the function returns True.

    Returns:
    - True if the graph has converged, False otherwise.
    """
    difference = np.sum(np.abs(current_graph - previous_graph))
    #return False
    return difference <= tolerance


def populate_edges(graph, c, beta, p, threshold = .5, max_iterations=100, sigma=1, plot_interval=0, pos=None):
    """Main function that uses all the above functions to fill the graph with edges."""
    i = 0
    stop_condition = False
    previous_graph = graph.copy()  # Initialize the previous graph to the initial graph

    output_folder = 'images'

    while i < max_iterations or not stop_condition:
        graph = add_vertex(graph, c, beta, p, threshold, sigma)
        stop_condition = check_convergence(graph, previous_graph)
        previous_graph = graph.copy()  # Update the previous graph

        if i % int(max_iterations*plot_interval) == 0:
            filename = os.path.join(output_folder, f'{i}.png')
            plot_graph_from_adjacency(graph, filename, pos=pos, title='iter: ' + str(i))

            #plot_graph_from_adjacency(graph,title='iter: '+str(i))

        i += 1
    return graph


# ER Graph for testing some functions

In [130]:
# Testing on a small graph
# test_graph2 = initialize_graph(5)
# populated_graph = populate_edges(test_graph2, 1e-3, 0.5, 1)
# populated_graph


# Test the function on a simple graph
#test_graph = initialize_graph( 5 )
#test_graph[0, 1] = 1
#test_graph[1, 0] = 1
#test_graph[1, 2] = 1
#test_graph[2, 1] = 1
#test_graph[2, 3] = 1
#test_graph[3, 2] = 1

#n = 10
#test_graph = generate_random_graph(n, .4)
#plot_graph_from_adjacency(test_graph)


In [131]:
#p = 0 # Order
#
#i = 4
#beta = 2e-3
#c = 1e-3
#
#print(degree_vertex(test_graph, i, p=p))
#print('soma degrees of vertex: ', get_sum_degrees(test_graph, i, p=p))
#
## Simple logistic regression function c / (1+beta*np.exp(sum_degrees))
#print('log reg value: ', logistic_regression(c=c, beta=beta, sum_degrees=get_sum_degrees(test_graph,i,p=p)))
#print('Edge or no edge: ', get_edge_logit(c = c, beta = beta, sum_degrees = get_sum_degrees(test_graph,i,p=p), threshold=0.5))

# Creating the graph using the logit model

In [144]:
graph = initialize_graph(50)
G = nx.Graph()
n = len(graph)
for i in range(n):
    for j in range(n):
        if graph[i][j] == 0 and i !=j :
            G.add_edge(i, j)
pos = nx.spring_layout(G)

####################
beta = 1e-5
c = 1e-4

p = 0 # Vizinhos
threshold = .5
sigma = 10
####################


graph = populate_edges(graph, c=c, beta=beta, p=p, threshold = threshold, max_iterations = int(300) ,sigma=sigma, plot_interval=0.01, pos=pos)

In [145]:
import os
import imageio

output_folder = 'images'

def create_gif_from_images(image_folder, gif_filename='graph_animation.gif', duration=100):
    images = []
    sorted_list = sorted(os.listdir('images'), key=lambda x: int(x.split('.')[0]))
    #for filename in os.listdir(image_folder):
    for filename in sorted_list:
        if filename.endswith(".png"):
            filepath = os.path.join(image_folder, filename)
            images.append(imageio.imread(filepath))

    imageio.mimsave(gif_filename, images, duration=duration)

# Call create_gif_from_images to create the GIF
create_gif_from_images(output_folder, 'graph_animation.gif')

/var/folders/ns/zy_pbb_s3591fwh2_q7z6g0h0000gn/T/ipykernel_27339/3160896743.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filepath))


In [106]:
#graph.sum(axis=1)

# Modifications on the logit model

* Added dynamical threshold
* Added damping factor

In [ ]:
def calculate_graph_density(graph):
    """Calculates the density of the graph."""
    n, m = graph.shape
    number_of_edges = np.sum(graph) / 2  # Divided by 2 because the graph is undirected
    max_possible_edges = n * (n - 1) / 2
    return number_of_edges / max_possible_edges

def dynamic_threshold(graph, base_threshold=0.3, alpha=0.5):
    """Calculates a dynamic threshold based on the graph's density."""
    density = calculate_graph_density(graph)
    return base_threshold + alpha * density

def refined_logistic_regression(c, beta, sum_degrees, alpha=1, gamma=0.05):
    """Refined logistic regression function with modified exponential term."""
    num     = c
    denom   = 1 + beta * np.exp(-alpha * sum_degrees + gamma * sum_degrees**2)
    return num / denom

def get_edge_logit_refined(c, beta, sum_degrees, threshold, alpha=1, gamma=0.01):
    """Decides if an edge should be added based on the refined logistic regression output and a threshold."""
    return 1 if refined_logistic_regression(c, beta, sum_degrees, alpha, gamma) >= threshold else 0

def add_vertex_with_dynamic_threshold(graph, c, beta, p, base_threshold=0.3, alpha=0.5, gamma=0.001, sigma=1):
    """Iterates over the graph's vertices and decides if an edge will be added using a dynamic threshold."""
    n, m = graph.shape
    for i in range(n):
        for j in range(m):
            if i != j and graph[i,j] == 0:
                sum_degrees_i = get_sum_degrees(graph, i, p)
                sum_degrees_j = get_sum_degrees(graph, j, p)
                sum_degrees = sum_degrees_i + sum_degrees_j + np.random.normal(0,sigma)
                threshold = dynamic_threshold(graph, base_threshold, alpha)
                #graph[i, j] = get_edge_logit(c, beta, sum_degrees, threshold)
                graph[i, j] = get_edge_logit_refined(c, beta, sum_degrees, threshold, alpha, gamma)
    return graph

def populate_edges_with_dynamic_threshold(graph, c, beta, p, base_threshold=0.3, alpha=0.5, max_iterations=10000):
    """Main function that uses all the above functions to fill the graph with edges using a dynamic threshold."""
    i = 0
    stop_condition = False
    previous_graph = graph.copy()  # Initialize the previous graph to the initial graph
    while i < max_iterations or not stop_condition:
        graph = add_vertex_with_dynamic_threshold(graph, c, beta, p, base_threshold, alpha)
        stop_condition = check_convergence(graph, previous_graph)
        previous_graph = graph.copy()  # Update the previous graph
        if i % int(max_iterations * 0.1) == 0:
            plot_graph_from_adjacency(graph)
        i += 1
    return graph


# Test the modified functions
test_graph = initialize_graph(10)
c = 1e-3
beta = 2e-3
p = 0
populated_graph_dynamic_threshold = populate_edges_with_dynamic_threshold(test_graph, c=c, beta=beta, p=p)


# Estimate parameters of the graph above

In [ ]:
def estimate_parameters():
    return 0